# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
file = 'data/city_weather_data.csv'
weather_df = pd.read_csv(file, index_col=0)
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,waipawa,-41.41,175.52,47.44,85,100,10.58,NZ,1632774571
1,butaritari,3.07,172.79,83.57,73,88,19.42,KI,1632774572
2,cherskiy,68.75,161.30,23.43,89,100,12.62,RU,1632774572
3,ushuaia,-54.80,-68.30,51.46,40,40,16.11,AR,1632774449
4,ribeira grande,38.52,-28.70,70.20,78,40,12.66,PT,1632774573
...,...,...,...,...,...,...,...,...,...
558,kamuli,0.95,33.12,64.09,93,45,3.51,UG,1632774798
559,tacuarembo,-31.73,-55.98,71.29,77,0,8.41,UY,1632774799
560,mangrol,21.12,70.12,78.58,87,100,1.48,IN,1632774799
561,lamesa,32.74,-101.95,91.08,23,37,11.74,US,1632774799


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [50]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Set Lat and Lng as locations and Humidity as the weight.
locations = weather_df[['Lat', 'Lng']]
weights = weather_df['Humidity']

# Create Map
fig = gmaps.figure()

# Add heatmap layer to map
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [43]:
# Drop Null values
clean_weather_df = weather_df.dropna()

# Narrow Cities
narrow_weather_df = clean_weather_df[clean_weather_df['Max Temp']<75]
narrow_weather_df = narrow_weather_df[narrow_weather_df['Max Temp']>70]
narrow_weather_df = narrow_weather_df[narrow_weather_df['Wind Speed']<10]
narrow_weather_df = narrow_weather_df[narrow_weather_df['Cloudiness']==0]

narrow_weather_df 

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
222,mizdah,31.45,12.98,73.53,32,0,5.26,LY,1632774664
311,marsa matruh,31.35,27.25,73.60,68,0,6.91,EG,1632774700
401,koungou,-12.73,45.20,74.91,83,0,0.00,YT,1632774736
433,erzin,36.96,36.20,74.12,67,0,2.51,TR,1632774751
491,mao,39.89,4.27,72.16,78,0,4.61,ES,1632774772
529,shakawe,-18.37,21.85,72.97,16,0,4.45,BW,1632774787
532,iguape,-24.71,-47.56,72.16,78,0,8.59,BR,1632774529
559,tacuarembo,-31.73,-55.98,71.29,77,0,8.41,UY,1632774799


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [45]:
# Create hotel_df
hotel_df = narrow_weather_df

# Url for call
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
# Set paramaters
params = {'key': g_key,
          'radius': 5000,
          'type':'lodging'}
locations = []

#Create new column for hotel names
hotel_df['Hotel Name'] = ''

# Build query URL
for x in range(len(hotel_df)):
    location = hotel_df.iloc[x,1:3]
    params['location'] = f'{location[0]},{location[1]}'
    request_url = hotel_response.url
    hotel_response = requests.get(url = url, params = params)
    request_url = hotel_response.url
    hotel_data = hotel_response.json()
    hotel_df.iloc[x,9] = hotel_data['results'][0]['name']

    #Display dataframe
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
222,mizdah,31.45,12.98,73.53,32,0,5.26,LY,1632774664,فندق بنيه تحتيه
311,marsa matruh,31.35,27.25,73.60,68,0,6.91,EG,1632774700,Beau Site
401,koungou,-12.73,45.20,74.91,83,0,0.00,YT,1632774736,Villa Netibor
433,erzin,36.96,36.20,74.12,67,0,2.51,TR,1632774751,Artemis Otel
491,mao,39.89,4.27,72.16,78,0,4.61,ES,1632774772,ARTIEM Capri
529,shakawe,-18.37,21.85,72.97,16,0,4.45,BW,1632774787,Ditlhapi Guest House
532,iguape,-24.71,-47.56,72.16,78,0,8.59,BR,1632774529,Hotel Pousada Nossa Senhora do Carmo
559,tacuarembo,-31.73,-55.98,71.29,77,0,8.41,UY,1632774799,Complejo San Fructuoso


In [52]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [53]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.marker_layer(locations, info_box_content=hotel_info))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))